In [ ]:
!git clone https://github.com/trokas/ai_primer.git

Cloning into 'ai_primer'...
remote: Enumerating objects: 2876, done.
remote: Counting objects: 100% (918/918), done.
remote: Compressing objects: 100% (387/387), done.
remote: Total 2876 (delta 485), reused 862 (delta 431), pack-reused 1958
Receiving objects: 100% (2876/2876), 109.02 MiB | 23.99 MiB/s, done.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from torch.nn import functional as F
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
x=np.load("ai_primer/flatland_train.npz")

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=(50, 50, 1)),
        layers.Conv2D(10, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.4),
        layers.Conv2D(10, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Dropout(0.4),
        layers.Conv2D(10, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Dropout(0.4),
        layers.Conv2D(10, kernel_size=(3, 3), activation="relu", padding="same"),
        layers.Dropout(0.4),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(80, activation="relu"),
        layers.Dense(5, activation="softmax")
    ]
)

opt=keras.optimizers.AdamW(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
data=tf.convert_to_tensor(torch.tensor(x["X"], dtype=torch.float32)/255)

In [ ]:
#Sutvarkome labels reiksmes ir paverciame labels i kategorini vektoriu

labels=x['y']
labels[labels!=0]-=2
labels=tf.convert_to_tensor(torch.tensor(labels, dtype=torch.int8))
labels=keras.utils.to_categorical(labels, 5)

In [ ]:
#Is 2000 paskutiniu paveiksleliu pagaminame 2000 augmented paveiksleliu. Augmented paveikslelis gaunamas dvigubai sumazinant paprasta 
#paveiksleli ir atsitiktineje vietoje uzdedant ji ant juodo fono.

import time

image_array=x["X"]
modified_data=[]
for i in range(8000, 10000):
    unmodified_image=image_array[i]
    offset=[int(j) for j in torch.randint(0, 25, (2,))]
    background=np.zeros((50, 50))
    for j in range(25):
        for k in range(25):
            background[j+offset[0]][k+offset[1]]=np.mean(unmodified_image[2*j:2*j+2, 2*k:2*k+2])
    modified_data.append(background)
    if i%100==0:
        print(i)

In [ ]:
modified_tensor=tf.convert_to_tensor(np.stack(modified_data))/255
modified_tensor=tf.cast(modified_tensor, tf.float32)

In [ ]:
#Pakeiciame paskutinius 2000 paveiksleliu ju augmented paveiksleliais. Kadangi augmented paveikslelyje objektas lieka tas pats, 
#labels vektoriaus keisti nereikia

final_data=tf.concat([data[:8000], modified_tensor], axis=0)

In [ ]:
for j in range(10):
    model.fit(final_data[:9500], labels[:9500], batch_size=100, epochs=10)
    print("*"*100)
    print(f"Epoch {(j+1)*10}")
    #tikriname generalization su nemodifikuotais paveiksleliais
    model.evaluate(data[9500:], labels[9500:])
    #tikriname generalization su modifikuotais paveiksleliais
    model.evaluate(modified_tensor[1500:], labels[9500:])
    print("*"*100)

In [ ]:
#Sumaziname learning rate geresniam konvergavimui

opt=keras.optimizers.AdamW(learning_rate=0.0001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
for j in range(25):
    model.fit(final_data[:9500], labels[:9500], batch_size=100, epochs=10)
    print("*"*100)
    print(f"Epoch {(j+1)*10}")
    #tikriname generalization su nemodifikuotais paveiksleliais
    model.evaluate(data[9500:], labels[9500:])
    #tikriname generalization su modifikuotais paveiksleliais
    model.evaluate(modified_tensor[1500:], labels[9500:])
    print("*"*100)

In [ ]:
model.save("model.h5")